### Using external data sources and PostgreSQL - Carriers
we want to get the carrier data into Python. And we want to do it without downloading the data to our machine using a web browser so we will read the csv directly from the web into Python.

#### Importing carrier data from the internet into Python
We will use pandas to read the file we found on the internet directly. 
First import pandas

In [4]:
# Import pandas
import pandas as pd

let the IDE help by typing in pd.read and then see what the autocomplete suggests

In [5]:
# pd.read_csv

In [6]:
# Set column names to 'carrier' and 'name'
carrier_columns = ['carrier' , 'name']

Next, we use the Pandas read csv function with the following url to import the data into Python:   
https://raw.githubusercontent.com/dannguyen/bts-transstats-t100-domestic-demo/master/data/lookup-tables/L_UNIQUE_CARRIERS.csv


In [7]:
# Import carriers data from the web using pandas
carriers = pd.read_csv('https://raw.githubusercontent.com/dannguyen/bts-transstats-t100-domestic-demo/master/data/lookup-tables/L_UNIQUE_CARRIERS.csv', # the location on the internet of the file we want to read
                     names= carrier_columns, # Sets the column names to the values in carrier_columns
                     skiprows = 1) # Skips the row with column names##very important


In [8]:
# Print the carriers dataframe
carriers

,carrier,name
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.
...,...,...
1560,ZW,Air Wisconsin Airlines Corp
1561,ZX,Air Georgian
1562,ZX (1),Airbc Ltd.
1563,ZY,Atlantic Gulf Airlines


#### Inserting carrier data into the database in Python
Now that we have the data in Python, all we have to do is write it into our SQL database. Before we do that we should make sure that our data is clean. Let's run some basic summary statistics before we move on.

In [9]:
# Print carriers info
carriers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   carrier  1564 non-null   object
 1   name     1565 non-null   object
dtypes: object(2)
memory usage: 24.6+ KB


checking for the NULL value

In [10]:
# Count NULL values
print(carriers.isnull().sum())

carrier    1
name       0
dtype: int64


In [11]:
# Find missing carrier code
# here we mask it
carriers[carriers['carrier'].isnull()]

,carrier,name
926,NaN,North American Airlines


Now, from your own sql module import your function to retrieve data into a list out of the postgres database :

In [12]:
# Import get_dataframe function from the sql module
from sql_functions import get_dataframe

Now we make a connection, we import engine

In [13]:
# Import get_engine from sql_functions.py. 
from sql_functions import get_engine
# create a variable called engine using the get_engine function
engine = get_engine()

In [14]:
# Set the schema 

schema = 'cgn_analytics_22_3' 
table_name = 'carriers_sj'

In [15]:
# we need psycopg2 for raising possible error message
import psycopg2

In [16]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        carriers.to_sql(name='carriers sj', # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema='cgn_analytics_22_3', # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

(psycopg2.OperationalError) could not translate host name "host" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)


To summarise, we added an external data source to our PostgreSQL database which extends our existing data and allows us to run even better analyses.   
